In [1]:
import tensorflow as tf 
import pandas as pd 
import numpy as np 

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [2]:
VOCAB_SIZE = 5000
EMBEDDING_DIM = 300
MAX_LEN = 20
TRUNC_TYPE = 'post'
PADDING_TYPE = 'post'
OOV_TOKEN = '<OOV>'

In [3]:
train_data = pd.read_csv('../data/train_mod.csv') # for training
test_data = pd.read_csv('../data/test_mod.csv') # for testing

In [4]:
# Make tokenizer and word_index
sentence_tokenizer = text.Tokenizer(num_words=VOCAB_SIZE, oov_token=OOV_TOKEN)
sentence_tokenizer.fit_on_texts(train_data.sentence.values)
word_index = sentence_tokenizer.word_index

In [5]:
import io

ft = io.open('../pretrained/cc.vi.300.vec', encoding='utf-8')

embeddings_index = {}
for line in ft:
    values = line.rstrip().split(' ')
    word = values[0]
    vector = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = vector

ft.close()

words_not_found = []
embedding_matrix = np.zeros((VOCAB_SIZE, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        embedding_matrix[i] = embedding_vector
    else:
        words_not_found.append(word)
        
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix, axis=1) == 0))

number of null word embeddings: 1144


In [6]:
# Tokenize sentence
train_sentence = sentence_tokenizer.texts_to_sequences(train_data.sentence.values) # Convert all word to sequence
train_sentence = sequence.pad_sequences(train_sentence, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE) # Pad each entry
test_sentence = sentence_tokenizer.texts_to_sequences(test_data.sentence.values) # Convert all word to sequence
test_sentence = sequence.pad_sequences(test_sentence, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE) # Pad each entry

In [7]:
# Categorize emotion
train_emotion = pd.get_dummies(train_data.emotion.values)
test_emotion = pd.get_dummies(test_data.emotion.values)

In [8]:
model = keras.Sequential([
    layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_LEN, trainable=False),
    # layers.SpatialDropout1D(.5),
    layers.Dropout(.5),
    layers.Bidirectional(layers.LSTM(128)),
    layers.Dense(64, activation='sigmoid'),
    layers.Dropout(.5),
    layers.Dense(6, activation='softmax')
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 300)           1500000   
_________________________________________________________________
dropout (Dropout)            (None, 20, 300)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 256)               439296    
_________________________________________________________________
dense (Dense)                (None, 64)                16448     
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 390       
Total params: 1,956,134
Trainable params: 456,134
Non-trainable params: 1,500,000
________________________________________

In [10]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

initial_epochs = 30

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=3)
mc=ModelCheckpoint('LSTMV2.h5', monitor='val_accuracy', mode='max', save_best_only=True,verbose=1) 
history = model.fit(train_sentence, train_emotion,batch_size=64, epochs=initial_epochs, validation_split=.1, verbose=1, callbacks=[es,mc])
# history = model.fit(train_sentence, train_emotion,batch_size=64, epochs=initial_epochs, validation_split=.1, verbose=1)
# model.evaluate(test_sentence, test_emotion)

Train on 4074 samples, validate on 453 samples
Epoch 1/30
4032/4074 [============================>.] - ETA: 0s - loss: 1.1481 - accuracy: 0.5615
Epoch 00001: val_accuracy improved from -inf to 0.52980, saving model to LSTMV2.h5
4074/4074 [==============================] - 20s 5ms/sample - loss: 1.1470 - accuracy: 0.5626 - val_loss: 1.1555 - val_accuracy: 0.5298
Epoch 2/30
4032/4074 [============================>.] - ETA: 0s - loss: 1.1077 - accuracy: 0.5813
Epoch 00002: val_accuracy improved from 0.52980 to 0.59161, saving model to LSTMV2.h5
4074/4074 [==============================] - 8s 2ms/sample - loss: 1.1098 - accuracy: 0.5805 - val_loss: 1.1104 - val_accuracy: 0.5916
Epoch 3/30
4032/4074 [============================>.] - ETA: 0s - loss: 1.0841 - accuracy: 0.5878
Epoch 00003: val_accuracy did not improve from 0.59161
4074/4074 [==============================] - 9s 2ms/sample - loss: 1.0845 - accuracy: 0.5874 - val_loss: 1.1156 - val_accuracy: 0.5717
Epoch 4/30
4032/4074 [=======

In [11]:
model.load_weights('LSTMV2.h5')
result = model.evaluate(test_sentence, test_emotion)
print(result)

564/564 [==============================] - 1s 1ms/sample - loss: 1.1131 - accuracy: 0.5780
[1.1131218350525443, 0.5780142]


In [12]:
sentence = 'Thật là kinh khủng'
# labels = ['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Other', 'Sadness', 'Surprise']
labels = ['Anger', 'Disgust', 'Enjoyment', 'Fear', 'Sadness', 'Surprise']
# labels = ['Negative', 'Neutral', 'Positive']
sentence = sentence_tokenizer.texts_to_sequences([sentence])
sentence = sequence.pad_sequences(sentence, maxlen=MAX_LEN, padding=PADDING_TYPE, truncating=TRUNC_TYPE)
pred = model.predict([sentence])
print(pred) 
print(labels[np.argmax(pred)])

[[0.00918274 0.0775242  0.55239874 0.01682269 0.06917091 0.27490067]]
Enjoyment
